In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.listdir('drive/Shareddrives/NLP - Music Success Prediction/Data')
os.chdir('drive/Shareddrives/NLP - Music Success Prediction/Data')

In [ ]:
import pandas as pd
df = pd.read_csv('Hit_2.csv')

In [ ]:
df.head()

,Unnamed: 0,SongID,Song_x,Performer_x,Lyrics_x,Hit_x,Cleaned_Lyrics
0,0,'65 Love AffairPaul Davis,'65 Love Affair,Paul Davis,I was a car hop\rYou were into be-bop\rYou san...,False,car hop bebop sang wop diddy wop diddy wop doo...
1,1,'Til My Baby Comes HomeLuther Vandross,'Til My Baby Comes Home,Luther Vandross,Theres a whole lot of girls\r\nmessin around\r...,False,whole lot girls messin around trying get thing...
2,2,'Til Summer Comes AroundKeith Urban,'Til Summer Comes Around,Keith Urban,Another long summer's come and gone\r\nI don't...,False,another long summer 's come gone know always e...
3,3,'Til You Do Me RightAfter 7,'Til You Do Me Right,After 7,I was in love with you\r\nAnd gave my heart to...,False,love gave heart best keep satisfied took love ...
4,4,'TilThe Angels,'Til,The Angels,"Due to copyright restrictions, we are not auth...",False,due copyright restrictions authorized display ...


In [ ]:
df.rename(columns={"Song_x": "Song", "Performer_x": "Performer","Lyrics_x":"Lyrics","Hit_x" : "Hit"}, inplace=True)

In [ ]:
df.head()

,Unnamed: 0,SongID,Song,Performer,Lyrics,Hit,Cleaned_Lyrics
0,0,'65 Love AffairPaul Davis,'65 Love Affair,Paul Davis,I was a car hop\rYou were into be-bop\rYou san...,False,car hop bebop sang wop diddy wop diddy wop doo...
1,1,'Til My Baby Comes HomeLuther Vandross,'Til My Baby Comes Home,Luther Vandross,Theres a whole lot of girls\r\nmessin around\r...,False,whole lot girls messin around trying get thing...
2,2,'Til Summer Comes AroundKeith Urban,'Til Summer Comes Around,Keith Urban,Another long summer's come and gone\r\nI don't...,False,another long summer 's come gone know always e...
3,3,'Til You Do Me RightAfter 7,'Til You Do Me Right,After 7,I was in love with you\r\nAnd gave my heart to...,False,love gave heart best keep satisfied took love ...
4,4,'TilThe Angels,'Til,The Angels,"Due to copyright restrictions, we are not auth...",False,due copyright restrictions authorized display ...


In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import pickle

In [ ]:
df = df[["Cleaned_Lyrics", "Hit"]]

In [ ]:
df.isna().sum()

,0
Cleaned_Lyrics,25
Hit,0


In [ ]:
df.dropna(inplace=True)

In [ ]:
# TF-IDF Vectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
tfidf_features = tfidf_vectorizer.fit_transform(df["Cleaned_Lyrics"])

KeyboardInterrupt: 

In [ ]:
svd = TruncatedSVD(n_components=300, random_state=42)
tfidf_reduced = svd.fit_transform(tfidf_features)

In [ ]:
combined_features = tfidf_reduced

In [ ]:
y = df["Hit"]
X_train, X_test, y_train, y_test = train_test_split(
    combined_features, y, test_size=0.2, random_state=42, stratify=y
)


# Random Classifier

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced")
rf_clf.fit(X_train, y_train)

y_pred = rf_clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.67      0.96      0.79      1776
        True       0.66      0.13      0.22       982

    accuracy                           0.67      2758
   macro avg       0.66      0.55      0.50      2758
weighted avg       0.67      0.67      0.58      2758



In [ ]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42, class_weight="balanced"),
                           param_grid, cv=5, scoring="f1", n_jobs=-1)
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)


Best parameters: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 100}


In [ ]:
y_pred_tuned = grid_search.best_estimator_.predict(X_test)
print(classification_report(y_test, y_pred_tuned))

              precision    recall  f1-score   support

       False       0.70      0.82      0.76      1776
        True       0.54      0.38      0.44       982

    accuracy                           0.66      2758
   macro avg       0.62      0.60      0.60      2758
weighted avg       0.65      0.66      0.65      2758



In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [ ]:
texts = df["Cleaned_Lyrics"].tolist()
labels = df["Hit"].tolist()

In [ ]:
max_words = 10000  # vocabulary size
max_len = 300      # maximum sequence length
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding="post", truncating="post")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42, stratify=labels)

NameError: name 'padded_sequences' is not defined

In [ ]:
model = Sequential([
    Embedding(max_words, 128, input_length=max_len),
    LSTM(128, return_sequences=True),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train, np.array(y_train), epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stop])

Epoch 1/10
276/276 ━━━━━━━━━━━━━━━━━━━━ 234s 824ms/step - accuracy: 0.6441 - loss: 0.6521 - val_accuracy: 0.6475 - val_loss: 0.6514
Epoch 2/10
276/276 ━━━━━━━━━━━━━━━━━━━━ 233s 844ms/step - accuracy: 0.6564 - loss: 0.6536 - val_accuracy: 0.6611 - val_loss: 0.6420
Epoch 3/10
276/276 ━━━━━━━━━━━━━━━━━━━━ 254s 918ms/step - accuracy: 0.6659 - loss: 0.6348 - val_accuracy: 0.6729 - val_loss: 0.6135
Epoch 4/10
276/276 ━━━━━━━━━━━━━━━━━━━━ 238s 832ms/step - accuracy: 0.6805 - loss: 0.5987 - val_accuracy: 0.6715 - val_loss: 0.6211
Epoch 5/10
276/276 ━━━━━━━━━━━━━━━━━━━━ 263s 834ms/step - accuracy: 0.7169 - loss: 0.5743 - val_accuracy: 0.6624 - val_loss: 0.6400
Epoch 6/10
276/276 ━━━━━━━━━━━━━━━━━━━━ 263s 839ms/step - accuracy: 0.7251 - loss: 0.5701 - val_accuracy: 0.6434 - val_loss: 0.6460


In [ ]:
loss, accuracy = model.evaluate(X_test, np.array(y_test))
print(f"Test Accuracy (CNN): {accuracy:.4f}")

87/87 ━━━━━━━━━━━━━━━━━━━━ 18s 207ms/step - accuracy: 0.6558 - loss: 0.6354
Test Accuracy (CNN): 0.6472


In [ ]:
y_prob = model.predict(X_test)

# Convert probabilities to class labels (assuming binary classification with threshold 0.5)
y_pred = (y_prob > 0.5).astype(int)


print(classification_report(y_test, y_pred))

87/87 ━━━━━━━━━━━━━━━━━━━━ 20s 222ms/step
              precision    recall  f1-score   support

       False       0.67      0.89      0.76      1776
        True       0.51      0.21      0.30       982

    accuracy                           0.65      2758
   macro avg       0.59      0.55      0.53      2758
weighted avg       0.61      0.65      0.60      2758



NameError: name 'X_train' is not defined

In [ ]:
!pip install autogluon


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. 

In [ ]:
import pandas as pd
from autogluon.tabular import TabularPredictor


# Ensure the "Hit" column is categorical
df["Hit"] = df["Hit"].astype(str)  # Convert to string for classification.

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [ ]:
predictor = TabularPredictor(label="Hit").fit(train_df, presets="best_quality")


No path specified. Models will be saved in: "AutogluonModels/ag-20250317_001617"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       11.00 GB / 12.67 GB (86.8%)
Disk Space Avail:   59.16 GB / 107.72 GB (54.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be f

In [ ]:
performance = predictor.evaluate(test_df)
print(performance)

{'accuracy': 0.7142857142857143, 'balanced_accuracy': 0.640155787637453, 'mcc': 0.33853821222572644, 'roc_auc': 0.7405806474445622, 'f1': 0.4863102998696219, 'precision': 0.6881918819188192, 'recall': 0.37600806451612906}


In [ ]:
predictions = predictor.predict(test_df.drop(columns=["Hit"]))
print(predictions)


6625     False
2382      True
6086     False
8283      True
4107     False
         ...  
2680     False
135      False
13361    False
11467    False
12940    False
Name: Hit, Length: 2758, dtype: object


In [ ]:
predictor.save("hit_song_predictor")

In [ ]:
import pickle

# Define the save path in Google Drive Shared Drive
drive_path = "/content/drive/Shareddrives/NLP - Music Success Prediction/Data/hit_song_predictor.pkl"

# Save the model as a .pkl file
with open(drive_path, "wb") as f:
    pickle.dump(predictor, f)

print(f"Model saved at: {drive_path}")

from google.colab import files

# Download the saved model from Google Drive
files.download(drive_path)



Model saved at: /content/drive/Shareddrives/NLP - Music Success Prediction/Data/hit_song_predictor.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
leaderboard = predictor.leaderboard(test_df, silent=True)
leaderboard.head()


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L2,0.722263,0.713923,accuracy,39.375848,357.440096,1489.166239,1.183946,7.202503,87.262685,2,True,14
1,RandomForestGini_BAG_L2,0.720812,0.721537,accuracy,39.276632,429.037300,1496.557583,1.084730,78.799707,94.654028,2,True,12
2,RandomForestEntr_BAG_L2,0.720450,0.722806,accuracy,39.104903,428.736796,1481.268147,0.913002,78.499203,79.364592,2,True,13
3,LightGBM_BAG_L1,0.718637,0.723169,accuracy,3.674795,1.878628,224.137913,3.674795,1.878628,224.137913,1,True,4
4,WeightedEnsemble_L2,0.718637,0.723169,accuracy,3.699325,1.880551,224.430007,0.024530,0.001923,0.292094,2,True,9


In [ ]:
leaderboard.to_csv("leaderboard.csv", index=False)
cv_save_path = "/content/drive/Shareddrives/NLP - Music Success Prediction/Data/leaderboard.csv"
files.download(cv_save_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from autogluon.tabular import TabularPredictor
import os

model_save_path = "/content/drive/Shareddrives/NLP - Music Success Prediction/Data/hit_song_predictor"

# Ensure the folder exists before saving
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

# Save the model
predictor.save(model_save_path)

print(f"Model successfully saved at: {model_save_path}")


Model successfully saved at: /content/drive/Shareddrives/NLP - Music Success Prediction/Data/hit_song_predictor


In [ ]:

import shutil
from google.colab import files

# Define zip file path
zip_model_path = "/content/hit_song_predictor.zip"

# Zip the saved model folder
shutil.make_archive(zip_model_path.replace('.zip', ''), 'zip', model_save_path)

# Download the zipped model
files.download(zip_model_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
leaderboard

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L2,0.722263,0.713923,accuracy,39.375848,357.440096,1489.166239,1.183946,7.202503,87.262685,2,True,14
1,RandomForestGini_BAG_L2,0.720812,0.721537,accuracy,39.276632,429.037300,1496.557583,1.084730,78.799707,94.654028,2,True,12
2,RandomForestEntr_BAG_L2,0.720450,0.722806,accuracy,39.104903,428.736796,1481.268147,0.913002,78.499203,79.364592,2,True,13
3,LightGBM_BAG_L1,0.718637,0.723169,accuracy,3.674795,1.878628,224.137913,3.674795,1.878628,224.137913,1,True,4
4,WeightedEnsemble_L2,0.718637,0.723169,accuracy,3.699325,1.880551,224.430007,0.024530,0.001923,0.292094,2,True,9
5,LightGBMXT_BAG_L2,0.717549,0.726976,accuracy,40.882283,352.761775,1542.650590,2.690382,2.524182,140.747036,2,True,10
6,CatBoost_BAG_L1,0.714286,0.721809,accuracy,1.223665,5.674837,645.960846,1.223665,5.674837,645.960846,1,True,7
7,LightGBM_BAG_L2,0.714286,0.727611,accuracy,40.914733,352.675981,1590.457466,2.722832,2.438388,188.553912,2,True,11
8,WeightedEnsemble_L3,0.714286,0.727611,accuracy,40.921026,352.677861,1591.120266,0.006293,0.001880,0.662799,3,True,15
9,ExtraTreesGini_BAG_L1,0.712835,0.704677,accuracy,1.543721,77.950861,157.183659,1.543721,77.950861,157.183659,1,True,8


In [31]:
print(predictor.model_best)

WeightedEnsemble_L3


In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
os.listdir()

['lyrics_success_prediction_ap.py',
 'labelled_success.csv',
 'Hit_1.csv',
 'Hit_2.csv',
 'AutogluonModels',
 'hit_song_predictor.pkl',
 'hit_song_predictor',
 'leaderboard.csv']

In [35]:
save_path = "content/drive/Shareddrives/NLP - Music Success Prediction/AG"
predictor.save(save_path)
print("Model saved to Google Drive!")

Model saved to Google Drive!


In [36]:
!ls /content/drive/Shareddrives/NLP\ -\ Music\ Success\ Prediction/

 Data		    'Jobs to Extract Lyrics'   Model_Training_Hit_2.ipynb
'Extracted Lyrics'   labelled_success.gsheet  'Music Success Classification Detail.gdoc'


In [37]:
predictor.save()

TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/content/drive/Shareddrives/NLP - Music Success Prediction/Data/AutogluonModels/ag-20250317_001617")
